In [56]:
import pandas as pd
pd.set_option('display.max_columns', None)

import requests
import json
import prettytable

In [192]:
blsSeries = pd.read_excel('../msaBlsSeries.xlsx')

In [129]:
months = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8,
          'September':9,'October':10,'November':11,'December':12}

In [88]:
uRateList = blsSeries['unemploymentRateSeries'].tolist()
uRateChunk1 = uRateList[:50]
areas = ','.join(uRateChunk1)

In [113]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=2075e7710bca44038c4abc07eecee9d5'
postSettings = {
                  "url": "https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=2075e7710bca44038c4abc07eecee9d5",
                  "method": "POST",
                  "headers": {
                    "Content-Type": "application/x-www-form-urlencoded"
                  },
                  "data": {
                    "seriesid": areas,
                    "startyear":"2017",
                    "endyear":"2020",
                  }
                }

response = requests.post(url, data = {"seriesid":areas,"startyear":"2018","endyear":"2020"})

In [114]:
json_data = json.loads(response.text)
seriesId = []
year = []
month = []
value = []

for series in json_data['Results']['series']:
    sid = series['seriesID']
    for item in series['data']:
        seriesId.append( sid )
        year.append( item['year'] )
        month.append ( item['periodName'] )
        value.append( item['value'] )
        
df = pd.DataFrame({'seriesId':seriesId,'month':month,'year':year,'value':value})
uRate = blsSeries[['MSA','unemploymentRateSeries']]

In [122]:
df = pd.merge(df,uRate,how='left',left_on='seriesId',right_on='unemploymentRateSeries')
df['Day'] = 1
df['Month'] = df.month.map(months)
df['date'] = pd.to_datetime(df[['Month','Day','year']])
df.sort_values(['seriesId','date'], inplace=True)
df['Month'] = df['month'].str[:3]
df['Date'] = df['Month'] + " " + df['year']
df['value'] = df['value'].astype(float)

data = df.pivot_table('value', ['Date', 'date'], 'MSA')
data.reset_index(inplace=True)
data.sort_values('date', inplace=True)
del data['date']

In [167]:
data.to_csv('deleteTest.csv', index=False)

# Repeat that process but with the total labor force

In [184]:
lfList = blsSeries['laborForceSeries'].tolist()
lfChunk1 = lfList[:50]
areas = ','.join( lfChunk1 )

In [185]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=2075e7710bca44038c4abc07eecee9d5'
postSettings = {
                  "url": "https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=2075e7710bca44038c4abc07eecee9d5",
                  "method": "POST",
                  "headers": {
                    "Content-Type": "application/x-www-form-urlencoded"
                  },
                  "data": {
                    "seriesid": areas,
                    "startyear":"2017",
                    "endyear":"2020",
                  }
                }

response = requests.post(url, data = {"seriesid":areas,"startyear":"2018","endyear":"2020"})

In [186]:
json_data = json.loads(response.text)
seriesId = []
year = []
month = []
value = []

for series in json_data['Results']['series']:
    sid = series['seriesID']
    for item in series['data']:
        seriesId.append( sid )
        year.append( item['year'] )
        month.append ( item['periodName'] )
        value.append( item['value'] )
        
df = pd.DataFrame({'seriesId':seriesId,'month':month,'year':year,'value':value})
lfRate = blsSeries[['MSA','laborForceSeries']]

In [187]:
df = pd.merge(df,lfRate,how='left',left_on='seriesId',right_on='laborForceSeries')
df['Day'] = 1


In [188]:
df.head()

,seriesId,month,year,value,MSA,laborForceSeries,Day
0,LAUMT011150000000006,October,2020,44790,"Anniston-Oxford-Jacksonville, AL",LAUMT011150000000006,1
1,LAUMT011150000000006,September,2020,45314,"Anniston-Oxford-Jacksonville, AL",LAUMT011150000000006,1
2,LAUMT011150000000006,August,2020,44829,"Anniston-Oxford-Jacksonville, AL",LAUMT011150000000006,1
3,LAUMT011150000000006,July,2020,44787,"Anniston-Oxford-Jacksonville, AL",LAUMT011150000000006,1
4,LAUMT011150000000006,June,2020,44000,"Anniston-Oxford-Jacksonville, AL",LAUMT011150000000006,1


In [189]:
df['Month'] = df.month.map(months)
df['date'] = pd.to_datetime(df[['Month','Day','year']])
df.sort_values(['seriesId','date'], inplace=True)
df['Month'] = df['month'].str[:3]
df['Date'] = df['Month'] + " " + df['year']
df['value'] = df['value'].astype(float)

data = df.pivot_table('value', ['Date', 'date'], 'MSA')
data.reset_index(inplace=True)
data.sort_values('date', inplace=True)
del data['date']

In [191]:
data.to_csv('deleteMeNow.csv', index=False)